# barra2-dl point data demo
This script uses barra2-dl to download BARRA2 AUS11 files for a list of variables. Individual data files will be 
saved as csv files in a cache folder, then merged into a single pandas dataframe. As a final step the wind ua and va 
components are converted to v and phi_met, before saving to a new csv file in an output folder.

The first step is to import the necessary python packages and modules from barra2_dl. Also from barra2_dl.globals we 
import some pre-configured default variables.

In [17]:
from datetime import datetime
from pathlib import Path

import barra2_dl
from barra2_dl.globals import barra2_var_wind_50m, barra2_var_wind_custom, BARRA2_AUS11_INDEX

In [18]:
print('BARRA2_AUS11_INDEX: ' + BARRA2_AUS11_INDEX.__str__())
print('barra2_var_wind_50m: ' + barra2_var_wind_50m.__str__())
print('barra2_var_wind_custom: ' + barra2_var_wind_custom.__str__())

BARRA2_AUS11_INDEX: ['time', 'station', 'latitude[unit="degrees_north"]', 'longitude[unit="degrees_east"]']
barra2_var_wind_50m: ['ua50m', 'va50m', 'ta50m']
barra2_var_wind_custom: ['ua50m', 'va50m', 'ua100m', 'va100m', 'ua150m', 'va150m', 'ta50m']


## Set variables
Set the cache and output folders.

In [30]:
cache_dir = r'cache'
output_dir = r'output'

Set the location point for downloading. This can either be set explicitly as a Dictionary, or using the 
pre-configured LatLonPoint class in barra2_dl.mapping module. Point data is downloaded to the nearest node. 

In [20]:
lat_lon_point = dict(lat=-23.5527472, lon=133.3961111)
from barra2_dl.mapping import LatLonPoint
lat_lon_point = LatLonPoint(lat=-23.5527472, lon=133.3961111)

print(lat_lon_point)

{'lat': -23.5527472, 'lon': 133.3961111}


Set start and end time for download.

In [21]:
start_datetime = datetime.strptime("2023-01-01T00:00:00Z", "%Y-%m-%dT%H:%M:%SZ")
end_datetime = datetime.strptime("2023-03-31T23:00:00Z", "%Y-%m-%dT%H:%M:%SZ")

Set output file custom name prefix to indicate a device or project location for the downloaded data. I.e. 
use project or location name.


In [22]:
fileout_prefix = "demo"

## Download point data
Use get_point_data to download closest node for the desired variables into the target cache folder.

In [31]:
barra2_dl.download.get_point_data(
    barra2_var_wind_50m,
    lat_lon_point,
    start_datetime,
    end_datetime,
    fileout_prefix = fileout_prefix,
    fileout_folder= cache_dir,
)

<demo_ua50m_20230101_20230131.csv> already exists in the folder <cache>
<demo_ua50m_20230201_20230228.csv> already exists in the folder <cache>
<demo_ua50m_20230301_20230331.csv> already exists in the folder <cache>
<demo_va50m_20230101_20230131.csv> already exists in the folder <cache>
<demo_va50m_20230201_20230228.csv> already exists in the folder <cache>
<demo_va50m_20230301_20230331.csv> already exists in the folder <cache>
<demo_ta50m_20230101_20230131.csv> already exists in the folder <cache>
<demo_ta50m_20230201_20230228.csv> already exists in the folder <cache>
<demo_ta50m_20230301_20230331.csv> already exists in the folder <cache>


## Combine data 
Merge downloaded csvs into a new dataframe. Optionally export merged data to a new csv. 

In [33]:
df_merged = barra2_dl.merge.merge_csvs_to_df(cache_dir, index_for_join=BARRA2_AUS11_INDEX)
print(df_merged.head())

Merged file: cache\demo_ta50m_20230201_20230228.csv
Merged file: cache\demo_ta50m_20230301_20230331.csv
Merged file: cache\demo_ua50m_20230101_20230131.csv
Merged file: cache\demo_ua50m_20230201_20230228.csv
Merged file: cache\demo_ua50m_20230301_20230331.csv
Merged file: cache\demo_va50m_20230101_20230131.csv
Merged file: cache\demo_va50m_20230201_20230228.csv
Merged file: cache\demo_va50m_20230301_20230331.csv
                   time                                 station  \
0  2023-01-01T00:00:00Z  GridPointRequestedAt[23.553S_133.396E]   
1  2023-01-01T01:00:00Z  GridPointRequestedAt[23.553S_133.396E]   
2  2023-01-01T02:00:00Z  GridPointRequestedAt[23.553S_133.396E]   
3  2023-01-01T03:00:00Z  GridPointRequestedAt[23.553S_133.396E]   
4  2023-01-01T04:00:00Z  GridPointRequestedAt[23.553S_133.396E]   

   latitude[unit="degrees_north"]  longitude[unit="degrees_east"]  \
0                          -23.54                          133.36   
1                          -23.54          

In [34]:
df_merged.to_csv(Path(output_dir) / f"{fileout_prefix}_merged_{start_datetime.strftime("%Y%m%d")}_{end_datetime.strftime("%Y%m%d")}.csv", index=False)

## Convert wind speed components
Using the merged dataframe, convert ua and va to v and phi_met, and export to new csv file

In [36]:
df_converted = barra2_dl.convert.convert_wind_components(df_merged)
print(df_converted.head())

Converting: ua50m[unit="m s-1"], va50m[unit="m s-1"]
                   time                                 station  \
0  2023-01-01T00:00:00Z  GridPointRequestedAt[23.553S_133.396E]   
1  2023-01-01T01:00:00Z  GridPointRequestedAt[23.553S_133.396E]   
2  2023-01-01T02:00:00Z  GridPointRequestedAt[23.553S_133.396E]   
3  2023-01-01T03:00:00Z  GridPointRequestedAt[23.553S_133.396E]   
4  2023-01-01T04:00:00Z  GridPointRequestedAt[23.553S_133.396E]   

   latitude[unit="degrees_north"]  longitude[unit="degrees_east"]  \
0                          -23.54                          133.36   
1                          -23.54                          133.36   
2                          -23.54                          133.36   
3                          -23.54                          133.36   
4                          -23.54                          133.36   

   ta50m[unit="K"]  ua50m[unit="m s-1"]  va50m[unit="m s-1"]  \
0       300.110962            -3.385254            -2.505127   
1

In [37]:
df_converted.to_csv(Path(output_dir) / f"{fileout_prefix}_converted_{start_datetime.strftime("%Y%m%d")}_{end_datetime.strftime("%Y%m%d")}.csv", index=False)

The merged and converted data is now ready to import into your favourite wind analysis program...